# Clustering Clinical Data in Python 

In this part of the practical we will be going through step by step how to use k-means on health records to find subtypes of a disease. 

In this tutorial we will use a real health dataset with [data for diabetic patients](https://archive.ics.uci.edu/ml/datasets/Diabetes+130-US+hospitals+for+years+1999-2008) in several US hospitals.
You can find a description of the variables via [this link](https://www.hindawi.com/journals/bmri/2014/781670/tab1/)

### Load Packages 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tableone import TableOne
from sklearn.cluster import KMeans
import warnings
import matplotlib.cm as cm
from sklearn import metrics
warnings.filterwarnings('ignore')


## Load and Inspect Data 

In [ ]:
data = pd.read_csv('diabetic_data.csv')
data.head()

In [ ]:
data.shape

## Cleaning Data 
This data has been cleaned in another script which you can find in this folder. 
<br>
First variables with high missingness were removed, these were weight, max_glu_serum, A1Cresult, payer_code and medical_specialty. 
<br>
Number_emergency was removed as over 90% of patients had 0. 
<br>
Then patients with either no race or gender information were removed (3 rows removed).
<br>
All rows with NA values are then removed (3711 rows removed).
<br>
Patients that are under 30 years old are also removed (2009 rows removed).
<br>
Duplicated encounters for each patient is removed leaving only the last encounter (28812 rows removed).

### Importing Clean Data 

In [ ]:
data_clean = pd.read_csv('diabetic_data_clean.csv')
data_clean.head()

In [ ]:
data_clean.shape

### Creating Table One 

In [ ]:
# list of columns to be included in tableone
columns = [u'race', u'gender', u'age',
       u'time_in_hospital', u'num_lab_procedures', u'num_procedures',
       u'num_medications', u'number_outpatient', 
       u'number_inpatient', u'number_diagnoses', u'metformin',
       u'glipizide', u'glyburide', u'insulin',
       u'change', u'diabetesMed', u'readmitted']

numeric = [u'time_in_hospital', u'num_lab_procedures', u'num_procedures',
       u'num_medications', u'number_outpatient', 
       u'number_inpatient', u'number_diagnoses']

# list of columns containing categorical variables
categorical = [u'race', u'gender', u'age', u'metformin', u'glipizide', u'glyburide', u'insulin', 
               u'change', u'diabetesMed', u'readmitted']

table_one = TableOne(data_clean, columns, categorical)
table_one

## Applying K Means 
To cluster the data there are the following steps
<br>
1) Preparing the data
<br>
2) Dimensionality reduction: PCA
<br>
3) Deciding cluster number: elbow plot 
<br>
4) Applying K-Means 
<br>
5) Characterising clusters 
<br>
6) Internal validation of clusters
<br>
7) External validation of clusters 

### Getting the data ready 
For PCA and K-means the data needs to be numerical, so we are going to select the numerical variables. 

We will also take a sample of the patients as we will run out of memory with a data set this large. 

This numerical data also needs to be scaled as if there variance in one variable is much larger than the others it will result in the principle componant corresponding to that variable. [Here is a nice link that explains this in more detail](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html#sphx-glr-auto-examples-preprocessing-plot-scaling-importance-py)

In [ ]:
pre_cluster_data = data_clean[numeric] # Selecting only the numeric data 

In [ ]:
np.random.seed(4) # setting random seed so random sample is repeatable 

sample_idx = np.random.randint(pre_cluster_data.shape[0],size=10000) 

pre_cluster_sample = pre_cluster_data.loc[sample_idx,:] # selecting a sample 
pre_cluster_sample.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() # gets z value of variable

pre_cluster_scaled = scaler.fit_transform(pre_cluster_sample)
pre_cluster_scaled

### Principle Components Analysis 
Now we will reduce the dimensions, as mentioned in the lecture this is to remove correlation between the variables.
In PCA we will find n-1 componants, ordered from the one which explains the most variance, to the least. We want to pick the number of componants that explains ~90% of the variance. 

We will be using the [pca function from scikit learn,](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). One of the parameters it takes is n_components which if greater than or equal to 1, it returns that number of components, or if less that 1 it returns the number of components that explain that proportion of the variance 

In [ ]:
from sklearn.decomposition import PCA
n_comp =  # Define components/variance returned here 
pca = PCA(n_components = n_comp)   
pca_res = pca.fit_transform(pre_cluster_scaled) # the .fit_transform applies PCA to the function and returns the transformed values 
pca_res_var = pca.explained_variance_ratio_ # returns the explained variance for each componant 

In [ ]:
pd.DataFrame(pca_res)


In [ ]:
pd.DataFrame(pca_res_var)

We can then visualise the fist two components of the data which have the two highest levels of variance. 
We want to plot the first component on the x axis and the second on the y. The function plt.scatter will create a scatter plot. 
Define the variables x_values and y_values which will be plotted in the scatter plot 

In [ ]:
x_values =  # Define the values for the x coordinates here (the first principle component)
y_values =  # Define the values for the y coordinates here (the second principle component)
plt.scatter(x_values, y_values)


### Finding Cluster Number 
Now we have our data set which has undergone dimensionality reduction, it is now time to apply K-Means. However before we do we need to find the number of clusters which gives us the best clustering solution. 
As mentioned in the lecture K-Means tries to minimize the squared error for each point for each cluster. This is also the squared euclidean distance between each point and the center of its corresponding cluster. We can sum this distance up for each point to each cluster centre to get a total for each cluster, then sum this up for each cluster. This, in scikit learn is called inertia. A low inertia means that all the points are very close to their assigned cluster centers and thus a better clustering solution. 
Lets run K-Means on our data set with two clusters and take a look at the inertia

[More information on the scikit learn kmeans function](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html)

In [ ]:
k = 2
kmeans = KMeans(n_clusters= k, init = 'k-means++' , random_state = 0).fit(pca_res)
kmeans.inertia_    

This value tells us nothing about how good a solution we found unless we compare it to the inertia found for different cluster numbers 
Below there is a for loop, complete the for loop so it loops a range of values of k (cluster number), runs k means and returns the inertia value. 
You will have to define the range of different cluster numbers you want to test. inertia_df is an empty data frame with the column names 'k' and 'inertia', you can define what is in a row by using df.loc[x] where x is the row label.  

In [ ]:
col_names = ["k", "inertia"] # Empty inertia array
inertia_df = pd.DataFrame(columns = col_names)
for k in range():# fill in range here
    # fill in for loop here

In [ ]:
inertia_df

the list below are the inertia values for the different cluster numbers, however it is hard to determine value of K to use based on this list. 

In [ ]:
inertia_df

### Elbow plot 
To work out the best value for K we can plot these values on a line graph with cluster number on the x axis and inertia on the y. As the cluster number increases the inertia will decrease, however it will not decrease linearly. The change in the inertia will start decreasing less between two cluster values which will form an "elbow" in the line graph (this is why its called an elbow plot). At the point where this elbow forms is what this method thinks is the best cluster number

#### Plotting the elbow graph 
Set values for cluster_N and inertia to plot the elbow plot 

In [ ]:
cluster_N =  # set  values for x coordinates, which is cluster numbers 
inertia =   # set values for y coordinates 
plt.plot(cluster_N, inertia)

What cluster number do you think the elbow plot suggests should be used? 
<br>
Do you think this is the best method we could use? 
<br>
What are the problems with it? 
<br>

### Elbow plot with a silhouette score
When K-Means Runs it aims to minimize the inertia,  we are now going to have a look at a metric that K-means does not try to minimize. For this we will use a silhouette score. This is using the mean silhouette coeffecient for each cluster result. The silhouette coefficient is a measure of how much each point belongs to its assigned cluster and it varies between -1 and 1, with values closer to 1 indicating better cluster assignment. 

First we will look at the silhouette score when k = 2 

In [ ]:
dist = metrics.pairwise_distances(pca_res) # finds a distance matrix using euclidian distance 
k = 2
kmeans = KMeans(n_clusters= k, init = 'k-means++' , random_state = 0).fit(pca_res)
silhouette_score = metrics.silhouette_score(dist, kmeans.labels_ , metric='precomputed') 
silhouette_score

Now fill in the for loop below to carry it out for a range of k 

In [ ]:
col_names_sil = ["k", "sil_score"] 
sil_df = pd.DataFrame(columns = col_names_sil) # Empty silhouette dataframe
for k in range():# fill in range here
    # fill in for loop here 

In [ ]:
sil_df

In [ ]:
cluster_N =  # set  values for x coordinates, which is cluster numbers 
sil_score =  # set values for y coordinates 
plt.plot(cluster_N, sil_score)

Does this give a clearer idea of what number to pick for k ? 


### Final K Means Results 
Take the value for K you found from the elbow plots and run K means, this will give us the final cluster results 

In [ ]:
final_k =  # fill in your final choice of cluster number here 
final_kmeans = KMeans(n_clusters = final_k, init = 'k-means++' , random_state = 0).fit(pca_res)


Finding out how many patients in each cluster 

In [ ]:
np.unique(final_kmeans.labels_, return_counts = True)

### Cluster Characterisation
Now we have assigned each patient to a cluster, we can see what variables that we used are high or low in each cluster.

In [ ]:
post_cluster_data = pre_cluster_sample

In [ ]:
post_cluster_data['cluster'] = 0
for i in range(final_k):
    post_cluster_data['cluster'][final_kmeans.labels_ == i] = (i + 1)

post_cluster_data.head()

In [ ]:
kmeans_table_one = TableOne(post_cluster_data, numeric, groupby = 'cluster')
kmeans_table_one

### Internal Validation measures 
#### PCA Plot 
We can also visualise our clusters using the pca plot we made earlier 

In [ ]:
LABEL_COLOUR_MAP = {0:'r', 1: 'g', 2: 'b', 3: 'c' , 4 : 'm'} # dictionary for colours 
label_colour = [LABEL_COLOUR_MAP[l] for l in final_kmeans.labels_]  

In [ ]:
plt.scatter(x_values, y_values, c = label_colour, alpha = 0.5)

Can you come to any conclusions about the results from the cluster analysis 

#### Silhouette plot 
Here we create a silhouette plot to visualise how much each point "belongs" to its assigned cluster. 

In [ ]:
silhouette_kmeans = metrics.silhouette_samples(dist,kmeans.labels_ , metric='precomputed')
silhouette_score = metrics.silhouette_score(dist, kmeans.labels_ , metric='precomputed')

In [ ]:
# Create a subplot with 1 row and 2 columns
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(14, 7)


 # The silhouette coefficient can range from -1, to 1 
ax.set_xlim([-1, 1])
# The (n_clusters+1)*10 is for inserting blank space between silhouette
# plots of individual clusters, to demarcate them clearly.
ax.set_ylim([0, len(pre_cluster_sample) + (final_k + 1) * 10])

labels = final_kmeans.labels_

y_lower = 10

for i in range(final_k):
    # Aggregate the silhouette scores for samples belonging to
    # cluster i, and sort them
    ith_cluster_silhouette_values = silhouette_kmeans[labels == i]
     
    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.spectral(float(i) / final_k)
    ax.fill_betweenx(np.arange(y_lower, y_upper),
                        0, ith_cluster_silhouette_values,
                        facecolor=color, edgecolor=color, alpha=0.7)

    # Label the silhouette plots with their cluster numbers at the middle
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i+1))

    # Compute the new y_lower for next plot
    y_lower = y_upper + 10  # 10 for the 0 samples

    
ax.set_title("Silhouette plot", fontsize=14, fontweight='bold')
ax.set_xlabel("The silhouette coefficient values", fontsize=14, fontweight='bold')
ax.set_ylabel("Cluster label", fontsize=14, fontweight='bold')

# The vertical line for average silhouette score of all the values
ax.axvline(x=silhouette_score, color="red", linestyle="--")

ax.set_yticks([])  # Clear the yaxis labels / ticks
ax.set_xticks([-0.4 ,-0.2, 0.0, 0.2, 0.4, 0.6, 0.8])
fig.show()

In [ ]:
silhouette_score

### External Validation 
Finding clusters which have differences in the variables we used to cluster is pretty much inevitable. The clusters are only relevent if there are differences in variables that were not included in the clustering. We have several catagorical variables in our original dataset that were not included in the analysis. Pick two or three variables that you think will be interesting to compare to see if they differ between clusters 

In [ ]:
outcome_factors = [] # Set the catagorical variables you want to examine here 
outcome_df = data_clean.loc[sample_idx ,outcome_factors] 

In [ ]:
outcome_df['cluster'] = 0
for i in range(final_k):
    outcome_df['cluster'][final_kmeans.labels_ == i] = (i + 1)

In [ ]:
outcome_table = TableOne(outcome_df , outcome_factors, outcome_factors, groupby = 'cluster' ,pval = True  )
outcome_table

Did you find a significant difference between your clusters? 

#### Other methods of external validation 
To test if the clusters you found are repeatable you can take another sample from the original data set, re-run the analysis and compare the results 

### Conclusion 
How would you characterise the clusters you found ?
<br>
Do you think K-Means found "good" clusters, why? 
<br>
Would you say the clusters you found are useful or clinically relevent ? 
<br>
What are the benefits of using this method ? 
<br>
What are the problems of using this method and how would you try to counteract them ? 